In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import poisson

In [2]:
data=pd.read_csv(r'C:\Users\enriq\Apple\matchesf1.csv')
data.columns = [c.lower() for c in data.columns]

In [3]:
data=data[data['comp']=='Premier League']

In [4]:
data['date']=pd.to_datetime(data['date'])
data["target"] = (data["result"] == "W").astype("int")
data['venue_code']=data['venue'].astype('category').cat.codes
data["opp_code"] = data["opponent"].astype("category").cat.codes
data["hour"] = data["time"].str.replace(":.+", "", regex=True).astype("int")
data["day_code"] = data["date"].dt.dayofweek

In [5]:
datax=pd.read_csv(r'C:\Users\enriq\Apple\last_wk.csv')

datax['Squad'].unique()

datax.columns

In [6]:
df_last=data[(data['round']=='Matchweek 27') &  (data['season']==2023)]

df_last['team'].unique()

In [7]:
mapping_dict = {
    'Arsenal': 'Arsenal',
    'Manchester City': 'Manchester City',
    'Manchester Utd': 'Manchester United',
    'Tottenham': 'Tottenham Hotspur',
    'Liverpool': 'Liverpool',
    'Newcastle Utd': 'Newcastle United',
    'Fulham': 'Fulham',
    'Brighton': 'Brighton and Hove Albion',
    'Brentford': 'Brentford',
    'Chelsea': 'Chelsea',
    'Aston Villa': 'Aston Villa',
    'Crystal Palace': 'Crystal Palace',
    'Wolves': 'Wolverhampton Wanderers',
    "Nott'ham Forest": 'Nottingham Forest',
    'Leicester City': 'Leicester City',
    'West Ham': 'West Ham United',
    'Leeds United': 'Leeds United',
    'Everton': 'Everton',
    'Southampton': 'Southampton',
    'Bournemouth': 'Bournemouth'
}

datax['Squad'] = datax['Squad'].map(mapping_dict)

datax['Squad'].unique()

In [8]:
xx=pd.merge(df_last,datax,left_on='team',right_on='Squad')

In [9]:
xx = xx.rename(columns={"squad": "team"})
xx

,unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,36,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Away,NaN,NaN,NaN,...,2.42,50.3,25.4,24.9,0.96,L W W W W,60196,Martinelli - 11,Aaron Ramsdale,NaN
1,89,2023-03-11,17:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,2.23,54.1,21.3,32.7,1.26,W W D W W,53219,Erling Haaland - 27,Ederson,NaN
2,146,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.96,39.7,31.4,8.4,0.34,W D W W L,73775,Marcus Rashford - 14,David de Gea,NaN
3,199,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Home,NaN,NaN,NaN,...,1.73,36.7,29.7,7.0,0.27,W L W W L,61669,Harry Kane - 18,Hugo Lloris,NaN
4,249,2023-03-11,12:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.68,46.0,33.7,12.4,0.49,W W D W W,53197,Mohamed Salah - 11,Alisson,NaN
5,296,2023-03-12,16:30,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.71,37.5,23.8,13.7,0.57,D D D L L,52241,Miguel Almirón - 10,Nick Pope,NaN
6,340,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.50,32.7,42.7,-10.0,-0.38,D W W D L,23431,Aleksandar Mitrović - 11,Bernd Leno,NaN
7,383,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.65,39.5,26.8,12.6,0.55,D W D L W,31501,Leandro Trossard - 7,Robert Sánchez,NaN
8,425,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.58,34.6,30.3,4.3,0.18,D W D D W,17076,Ivan Toney - 15,David Raya,NaN
9,474,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.36,29.8,30.8,-1.0,-0.04,D D L L W,39969,Kai Havertz - 5,Kepa Arrizabalaga,NaN


In [10]:
xx['xG']=xx['xG']/(xx['W']+xx['D']+xx['L'])

In [11]:
xx['xGA']=xx['xGA']/(xx['W']+xx['D']+xx['L'])
xx['xGD']=xx['xGD']/(xx['W']+xx['D']+xx['L'])

In [12]:
xx

,unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,36,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Away,NaN,NaN,NaN,...,2.42,1.934615,0.976923,0.957692,0.96,L W W W W,60196,Martinelli - 11,Aaron Ramsdale,NaN
1,89,2023-03-11,17:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,2.23,2.080769,0.819231,1.257692,1.26,W W D W W,53219,Erling Haaland - 27,Ederson,NaN
2,146,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.96,1.588000,1.256000,0.336000,0.34,W D W W L,73775,Marcus Rashford - 14,David de Gea,NaN
3,199,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Home,NaN,NaN,NaN,...,1.73,1.411538,1.142308,0.269231,0.27,W L W W L,61669,Harry Kane - 18,Hugo Lloris,NaN
4,249,2023-03-11,12:30,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.68,1.840000,1.348000,0.496000,0.49,W W D W W,53197,Mohamed Salah - 11,Alisson,NaN
5,296,2023-03-12,16:30,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.71,1.562500,0.991667,0.570833,0.57,D D D L L,52241,Miguel Almirón - 10,Nick Pope,NaN
6,340,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,NaN,NaN,NaN,...,1.50,1.257692,1.642308,-0.384615,-0.38,D W W D L,23431,Aleksandar Mitrović - 11,Bernd Leno,NaN
7,383,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.65,1.717391,1.165217,0.547826,0.55,D W D L W,31501,Leandro Trossard - 7,Robert Sánchez,NaN
8,425,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.58,1.441667,1.262500,0.179167,0.18,D W D D W,17076,Ivan Toney - 15,David Raya,NaN
9,474,2023-03-11,15:00,Premier League,Matchweek 27,Sat,Away,NaN,NaN,NaN,...,1.36,1.192000,1.232000,-0.040000,-0.04,D D L L W,39969,Kai Havertz - 5,Kepa Arrizabalaga,NaN


In [13]:
xx.drop(columns=["result", "gf", "ga", "opponent", "xg", "xga", "poss", "attendance", "captain", "formation", "referee"], inplace=True)


data[data['date']>'2023-03-01']

merged_df = pd.merge(data, xx, on=['date', 'team'], suffixes=('_x', '_y'))
merged_df['columna_con_valores_faltantes'] = merged_df['columna_con_valores_faltantes_x'].fillna(merged_df['columna_con_valores_faltantes_y'])


In [14]:
data.columns

Index(['unnamed: 0', 'date', 'time', 'comp', 'round', 'day', 'venue', 'result',
       'gf', 'ga', 'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain',
       'formation', 'referee', 'match report', 'notes', 'season', 'team',
       'target', 'venue_code', 'opp_code', 'hour', 'day_code'],
      dtype='object')

In [15]:
xx.columns

Index(['unnamed: 0', 'date', 'time', 'comp', 'round', 'day', 'venue',
       'match report', 'notes', 'season', 'team', 'target', 'venue_code',
       'opp_code', 'hour', 'day_code', 'Unnamed: 0', 'Rk', 'Squad', 'MP', 'W',
       'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'xG', 'xGA', 'xGD',
       'xGD/90', 'Last 5', 'Attendance', 'Top Team Scorer', 'Goalkeeper',
       'Notes'],
      dtype='object')

In [16]:
merged_df = pd.merge(data, xx, on=['unnamed: 0', 'date', 'time', 'comp', 'round', 'day', 'venue', 'match report', 'notes', 'season', 'team', 'target', 'venue_code', 'opp_code', 'hour', 'day_code'], how='left')
merged_df

,unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,3796,2020-07-07,18:00,Premier League,Matchweek 34,Tue,Away,L,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3036,3797,2020-07-11,12:30,Premier League,Matchweek 35,Sat,Home,L,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3037,3798,2020-07-14,20:15,Premier League,Matchweek 36,Tue,Away,L,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3038,3799,2020-07-18,17:30,Premier League,Matchweek 37,Sat,Home,L,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



merged_df['GF'] = merged_df['xg'].fillna(merged_df['xG'])
merged_df['GA'] = merged_df['xga'].fillna(merged_df['xGA'])
merged_df.drop(['gf', 'ga'], axis=1, inplace=True) # eliminar las columnas que ya no se necesitan


In [17]:
merged_df['xg'] = merged_df['xg'].fillna(merged_df['xG'])
merged_df['xga'] = merged_df['xga'].fillna(merged_df['xGA'])


In [18]:
columns=['date', 'time', 'comp', 'round', 'day', 'venue', 'opponent', 'xg', 'xga', 'season', 'team', 'target', 'venue_code', 'opp_code', 'hour', 'day_code']
merged_df=merged_df[columns]

In [19]:
xx=merged_df.dropna()
xx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 3039
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        2800 non-null   datetime64[ns]
 1   time        2800 non-null   object        
 2   comp        2800 non-null   object        
 3   round       2800 non-null   object        
 4   day         2800 non-null   object        
 5   venue       2800 non-null   object        
 6   opponent    2800 non-null   object        
 7   xg          2800 non-null   float64       
 8   xga         2800 non-null   float64       
 9   season      2800 non-null   int64         
 10  team        2800 non-null   object        
 11  target      2800 non-null   int32         
 12  venue_code  2800 non-null   int8          
 13  opp_code    2800 non-null   int8          
 14  hour        2800 non-null   int32         
 15  day_code    2800 non-null   int64         
dtypes: datetime64[ns](1), fl

In [20]:
xx=xx.sort_values(by='date')
xx

,date,time,comp,round,day,venue,opponent,xg,xga,season,team,target,venue_code,opp_code,hour,day_code
2280,2019-08-09,20:00,Premier League,Matchweek 1,Fri,Home,Norwich City,1.800000,0.900000,2020,Liverpool,1,1,16,20,4
3002,2019-08-09,20:00,Premier League,Matchweek 1,Fri,Away,Liverpool,0.900000,1.800000,2020,Norwich City,0,0,12,20,4
2622,2019-08-10,15:00,Premier League,Matchweek 1,Sat,Home,Southampton,0.900000,1.200000,2020,Burnley,1,1,19,15,5
2584,2019-08-10,15:00,Premier League,Matchweek 1,Sat,Away,Bournemouth,1.300000,1.300000,2020,Sheffield United,0,0,2,15,5
2698,2019-08-10,15:00,Premier League,Matchweek 1,Sat,Away,Crystal Palace,1.100000,0.900000,2020,Everton,0,0,7,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,2023-03-12,16:30,Premier League,Matchweek 27,Sun,Home,Wolves,1.562500,0.991667,2023,Newcastle United,0,1,24,16,6
254,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,Arsenal,1.257692,1.642308,2023,Fulham,0,1,0,14,6
595,2023-03-12,14:00,Premier League,Matchweek 27,Sun,Home,Aston Villa,1.252000,1.200000,2023,West Ham United,0,1,1,14,6
482,2023-03-12,16:30,Premier League,Matchweek 27,Sun,Away,Newcastle Utd,0.957692,1.426923,2023,Wolverhampton Wanderers,0,0,15,16,6


In [21]:
train = xx[xx["date"] < '2023-01-01']
test = xx[(xx["date"] > '2023-01-01')& (xx["date"]<'2023-03-11')]
match_week=xx[xx["date"] > '2023-03-10']

In [24]:
match_week.shape

(20, 16)

In [25]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

predictors = ["venue_code", "opp_code",'poss','xg','xga',"hour", "day_code"]
rf.fit(train[predictors], train["target"])

In [26]:
predictors = ["venue_code", "opp_code","hour", "day_code",'xg','xga']
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [27]:
preds = rf.predict(test[predictors])
match_week_preds=rf.predict(match_week[predictors])
accuracy= accuracy_score(test["target"], preds)
accuracy

0.75

In [28]:
preds

array([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0])

In [29]:
match_week_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0])

In [34]:
match_week_preds_df=pd.DataFrame(dict(predicted=match_week_preds))

In [37]:
datax['predicion']=match_week_preds_df

In [38]:
datax

,Unnamed: 0,Rk,Squad,MP,W,D,L,GF,GA,GD,...,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,predicion
0,0,1,Arsenal,26,20,3,3,59,25,34,...,50.3,25.4,24.9,0.96,L W W W W,60196,Martinelli - 11,Aaron Ramsdale,NaN,0
1,1,2,Manchester City,26,18,4,4,66,25,41,...,54.1,21.3,32.7,1.26,W W D W W,53219,Erling Haaland - 27,Ederson,NaN,0
2,2,3,Manchester United,25,15,4,6,41,35,6,...,39.7,31.4,8.4,0.34,W D W W L,73775,Marcus Rashford - 14,David de Gea,NaN,0
3,3,4,Tottenham Hotspur,26,14,3,9,46,36,10,...,36.7,29.7,7.0,0.27,W L W W L,61669,Harry Kane - 18,Hugo Lloris,NaN,0
4,4,5,Liverpool,25,12,6,7,47,28,19,...,46.0,33.7,12.4,0.49,W W D W W,53197,Mohamed Salah - 11,Alisson,NaN,0
5,5,6,Newcastle United,24,10,11,3,35,17,18,...,37.5,23.8,13.7,0.57,D D D L L,52241,Miguel Almirón - 10,Nick Pope,NaN,0
6,6,7,Fulham,26,11,6,9,38,34,4,...,32.7,42.7,-10.0,-0.38,D W W D L,23431,Aleksandar Mitrović - 11,Bernd Leno,NaN,0
7,7,8,Brighton and Hove Albion,23,11,5,7,43,29,14,...,39.5,26.8,12.6,0.55,D W D L W,31501,Leandro Trossard - 7,Robert Sánchez,NaN,0
8,8,9,Brentford,24,9,11,4,40,32,8,...,34.6,30.3,4.3,0.18,D W D D W,17076,Ivan Toney - 15,David Raya,NaN,1
9,9,10,Chelsea,25,9,7,9,24,25,-1,...,29.8,30.8,-1.0,-0.04,D D L L W,39969,Kai Havertz - 5,Kepa Arrizabalaga,NaN,0


In [30]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
combined

,actual,predicted
168,0,0
321,1,0
549,0,1
207,0,0
663,0,0
...,...,...
481,1,0
518,0,0
100,0,0
176,1,1


In [31]:
combined.sum()

actual       63
predicted    57
dtype: int64

In [32]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,87,18
1,24,39


In [33]:
error=precision_score(test["target"], preds)
error

0.6842105263157895

In [ ]:
grouped_matches = df.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [ ]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols=['xg','xga']

new_cols = [f"{c}_rolling" for c in cols]
rolling_averages(group, cols, new_cols)

In [ ]:
rolling_averages

In [ ]:
matches_rolling = df.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

In [ ]:
xx=matches_rolling.groupby(['team']).mean()

In [ ]:
xx

In [ ]:
type(xx)

In [ ]:
xx.index

In [ ]:
def predict_points(home, away):
    if home in xx.index and away in xx.index:
        # goals_scored * goals_conceded
        lamb_home = xx.at[home,'xg'] * xx.at[away,'xga']
        lamb_away = xx.at[away,'xg'] * xx.at[home,'xga']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home)
    else:
        return (0, 0)

In [ ]:
matches_rolling['home_points']=matches_rolling.apply(lambda x:predict_points(x['team'],x['opponent']),axis=1)
matches_rolling

In [ ]:
predict_points('Arsenal','Manchester City')

In [ ]:
predict_points('Liverpool', 'Manchester United')

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:

def make_predictions(data, predictors):
    train = data[data["date"] < '2023-01-01']
    test = data[data["date"] > '2023-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [ ]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
error

In [ ]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [ ]:

combined.head(22)

In [ ]:
def predict_points(home, away):
    if home in xx.index and away in xx.index:
        # goals_scored * goals_conceded
        lamb_home = xx.at[home,'xg'] * xx.at[away,'xga']
        lamb_away = xx.at[away,'xg'] * xx.at[home,'xga']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home)
    else:
        return (0, 0)